**Outline:**
* Problem: Gaussian KDE with h chosen by normal rule over-smooths, especially when more than one mode.
    + Mention Sheather (2004) text and put an example
* Alternative bandwidth selectors
    + Cross-Validation (for example, LSCV or BCV). They often result in wiggly estimates!
    + Plug-in estimators (Sheather-Jones and Improved-Sheather-Jones). Show when they really help (bimodal, for example), but also shown when they are also very wiggly! (leptokurtic, for example).
* Some examples (when they help and when they fail) and final remarks (that SJ / ISJ are recommended).